In [1]:
print('hello')

hello


<h1>Web Scraping</h1>

In [3]:
import requests
from bs4 import BeautifulSoup
# Wikipedia topics to scrape
topics = [
    #"ChatGPT",
    "Elon_Musk",
    "NVIDIA",
    "Artificial_intelligence",
    "Computer_science"
]
# requests_results = requests.get('https://en.wikipedia.org/wiki/ChatGPT')

In [4]:
# Function to scrape Wikipedia
def scrape_wikipedia(topic):
    url = f"https://en.wikipedia.org/wiki/{topic}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    paragraphs = []
    for paragraph in soup.select('p'):
        paragraphs.append(paragraph.get_text())
    
    return "\n".join(paragraphs)

In [5]:
for topic in topics:
    print(f"Scraping {topic}...")
    raw_text = scrape_wikipedia(topic)
    
    # Save to file
    filename = f"{topic.lower()}.txt"
    with open(filename, "w", encoding="utf-8") as file:
        for paragraph in raw_text.split("\n"):
            file.write(paragraph + "\n\n")
    print(f"Saved: {filename}")

print("Scraping complete!")

Scraping Elon_Musk...
Saved: elon_musk.txt
Scraping NVIDIA...
Saved: nvidia.txt
Scraping Artificial_intelligence...
Saved: artificial_intelligence.txt
Scraping Computer_science...
Saved: computer_science.txt
Scraping complete!


<h1>Text preprocessing</h1>

<h2>Import file and library</h2>

In [7]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer, LancasterStemmer
from collections import Counter
import spacy
import os
from collections import Counter

In [33]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     -- ------------------------------------- 0.8/12.8 MB 5.6 MB/s eta 0:00:03
     ---------- ----------------------------- 3.4/12.8 MB 9.6 MB/s eta 0:00:01
     ------------------ --------------------- 5.8/12.8 MB 10.4 MB/s eta 0:00:01
     -------------------------- ------------- 8.4/12.8 MB 10.8 MB/s eta 0:00:01
     -------------------------------- ------ 10.7/12.8 MB 11.2 MB/s eta 0:00:01
     --------------------------------------  12.6/12.8 MB 11.3 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 10.6 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [14]:
nlp = spacy.load("en_core_web_sm")

In [8]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
# Get list of all .txt files (assuming these are your scraped Wikipedia files)
txt_files = [f for f in os.listdir() if f.endswith('.txt')]
print("Found files:", txt_files)

# Read each file into a dictionary
data = {}
for filename in txt_files:
    with open(filename, "r", encoding="utf-8") as file:
        data[filename] = file.read()

Found files: ['artificial_intelligence.txt', 'chatgpt_wikipedia.txt', 'computer_science.txt', 'elon_musk.txt', 'nvidia.txt']


<h2>Tokenize</h2>

In [9]:
# Tokenize the text for each file
tokenized_data = {}
for filename, text in data.items():
    tokens = word_tokenize(text)
    tokenized_data[filename] = tokens
    print(f"Tokenized Words for {filename} (sample):", tokens[:10])

Tokenized Words for artificial_intelligence.txt (sample): ['Artificial', 'intelligence', '(', 'AI', ')', 'refers', 'to', 'the', 'capability', 'of']
Tokenized Words for chatgpt_wikipedia.txt (sample): ['ChatGPT', 'is', 'a', 'generative', 'artificial', 'intelligence', 'chatbot', 'developed', 'by', 'OpenAI']
Tokenized Words for computer_science.txt (sample): ['Computer', 'science', 'is', 'the', 'study', 'of', 'computation', ',', 'information', ',']
Tokenized Words for elon_musk.txt (sample): ['Elon', 'Reeve', 'Musk', '(', '/ˈiːlɒn/', 'EE-lon', ';', 'born', 'June', '28']
Tokenized Words for nvidia.txt (sample): ['Nvidia', 'Corporation', '[', 'a', ']', '(', '/ɛnˈvɪdiə/', 'en-VID-ee-ə', ')', 'is']


<h2>Casefold</h2>

In [10]:
# Convert tokens to lowercase for each file
casefolded_data = {}
for filename, tokens in tokenized_data.items():
    tokens_lower = [word.lower() for word in tokens]
    casefolded_data[filename] = tokens_lower
    print(f"Casefolded Words for {filename} (sample):", tokens_lower[:10])

Casefolded Words for artificial_intelligence.txt (sample): ['artificial', 'intelligence', '(', 'ai', ')', 'refers', 'to', 'the', 'capability', 'of']
Casefolded Words for chatgpt_wikipedia.txt (sample): ['chatgpt', 'is', 'a', 'generative', 'artificial', 'intelligence', 'chatbot', 'developed', 'by', 'openai']
Casefolded Words for computer_science.txt (sample): ['computer', 'science', 'is', 'the', 'study', 'of', 'computation', ',', 'information', ',']
Casefolded Words for elon_musk.txt (sample): ['elon', 'reeve', 'musk', '(', '/ˈiːlɒn/', 'ee-lon', ';', 'born', 'june', '28']
Casefolded Words for nvidia.txt (sample): ['nvidia', 'corporation', '[', 'a', ']', '(', '/ɛnˈvɪdiə/', 'en-vid-ee-ə', ')', 'is']


<h2>Remove Stopwords</h2>

In [11]:
# Remove punctuation and stopwords for each file
stop_words = set(stopwords.words('english'))
filtered_data = {}
for filename, tokens in casefolded_data.items():
    filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    filtered_data[filename] = filtered_tokens
    print(f"Filtered Tokens for {filename} (sample):", filtered_tokens[:10])

print("A sample of stopwords:", list(stop_words)[:10])

Filtered Tokens for artificial_intelligence.txt (sample): ['artificial', 'intelligence', 'ai', 'refers', 'capability', 'computational', 'systems', 'perform', 'tasks', 'typically']
Filtered Tokens for chatgpt_wikipedia.txt (sample): ['chatgpt', 'generative', 'artificial', 'intelligence', 'chatbot', 'developed', 'openai', 'launched', '2022', 'currently']
Filtered Tokens for computer_science.txt (sample): ['computer', 'science', 'study', 'computation', 'information', 'automation', '1', '2', '3', 'computer']
Filtered Tokens for elon_musk.txt (sample): ['elon', 'reeve', 'musk', 'born', 'june', '28', '1971', 'businessman', 'known', 'key']
Filtered Tokens for nvidia.txt (sample): ['nvidia', 'corporation', 'american', 'multinational', 'corporation', 'technology', 'company', 'headquartered', 'santa', 'clara']
A sample of stopwords: ['again', 'it', 'all', 'so', 'been', 'him', 'just', 'itself', 'on', 'as']


<h2>Stemming</h2>

In [12]:
# Apply stemming using LancasterStemmer for each file
lancaster = LancasterStemmer()
stemmed_data = {}
for filename, tokens in filtered_data.items():
    stemmed_tokens = [lancaster.stem(word) for word in tokens]
    stemmed_data[filename] = stemmed_tokens
    print(f"Stemmed Tokens for {filename} (sample):", stemmed_tokens[:10])

Stemmed Tokens for artificial_intelligence.txt (sample): ['art', 'intellig', 'ai', 'ref', 'cap', 'comput', 'system', 'perform', 'task', 'typ']
Stemmed Tokens for chatgpt_wikipedia.txt (sample): ['chatgpt', 'gen', 'art', 'intellig', 'chatbot', 'develop', 'opena', 'launch', '2022', 'cur']
Stemmed Tokens for computer_science.txt (sample): ['comput', 'sci', 'study', 'comput', 'inform', 'autom', '1', '2', '3', 'comput']
Stemmed Tokens for elon_musk.txt (sample): ['elon', 'reev', 'musk', 'born', 'jun', '28', '1971', 'businessm', 'known', 'key']
Stemmed Tokens for nvidia.txt (sample): ['nvid', 'corp', 'am', 'multin', 'corp', 'technolog', 'company', 'headquart', 'sant', 'clar']


<h2>Lemmatizing</h2>

In [15]:
# Apply lemmatization using spaCy for each file (on the filtered tokens)
lemmatized_data = {}
for filename, tokens in filtered_data.items():
    doc = nlp(" ".join(tokens))
    lemmatized_tokens = [token.lemma_ for token in doc if token.is_alpha]
    lemmatized_data[filename] = lemmatized_tokens
    print(f"Lemmatized Tokens for {filename} (sample):", lemmatized_tokens[:10])

Lemmatized Tokens for artificial_intelligence.txt (sample): ['artificial', 'intelligence', 'ai', 'refer', 'capability', 'computational', 'system', 'perform', 'task', 'typically']
Lemmatized Tokens for chatgpt_wikipedia.txt (sample): ['chatgpt', 'generative', 'artificial', 'intelligence', 'chatbot', 'develop', 'openai', 'launch', 'currently', 'base']
Lemmatized Tokens for computer_science.txt (sample): ['computer', 'science', 'study', 'computation', 'information', 'automation', 'computer', 'science', 'span', 'theoretical']
Lemmatized Tokens for elon_musk.txt (sample): ['elon', 'reeve', 'musk', 'bear', 'june', 'businessman', 'know', 'key', 'role', 'tesla']
Lemmatized Tokens for nvidia.txt (sample): ['nvidia', 'corporation', 'american', 'multinational', 'corporation', 'technology', 'company', 'headquarter', 'santa', 'clara']


In [16]:
# Build a Bag of Words representation for each file
bow_data = {}
for filename, tokens in filtered_data.items():
    bow = Counter(tokens)
    bow_data[filename] = bow
    print(f"Most Common Tokens in {filename}:", bow.most_common(10))

Most Common Tokens in artificial_intelligence.txt: [('ai', 227), ('intelligence', 65), ('learning', 59), ('used', 57), ('data', 56), ('artificial', 45), ('machine', 41), ('human', 35), ('research', 35), ('problems', 34)]
Most Common Tokens in chatgpt_wikipedia.txt: [('chatgpt', 192), ('openai', 75), ('2023', 69), ('ai', 57), ('users', 31), ('2024', 24), ('used', 23), ('model', 22), ('could', 22), ('one', 21)]
Most Common Tokens in computer_science.txt: [('computer', 90), ('science', 66), ('software', 26), ('engineering', 23), ('systems', 22), ('data', 21), ('computing', 20), ('information', 19), ('theory', 18), ('computation', 17)]
Most Common Tokens in elon_musk.txt: [('musk', 184), ('company', 34), ('tesla', 30), ('spacex', 19), ('2024', 18), ('elon', 17), ('billion', 17), ('twitter', 16), ('2022', 16), ('million', 16)]
Most Common Tokens in nvidia.txt: [('nvidia', 216), ('company', 44), ('announced', 31), ('graphics', 28), ('gpus', 26), ('huang', 24), ('2024', 21), ('ai', 19), ('gpu

<h2>Feature extraction</h2>

In [17]:
from collections import Counter

# Build a mapping of word -> {document_name: count} for each document
word_document_counts = {}
for doc, tokens in filtered_data.items():
    token_counts = Counter(tokens)
    for token, count in token_counts.items():
        if token not in word_document_counts:
            word_document_counts[token] = {}
        word_document_counts[token][doc] = count

# Optionally, build a summary for each word
word_occurrence_summary = {}
for word, doc_dict in word_document_counts.items():
    num_docs = len(doc_dict)
    total_occurrences = sum(doc_dict.values())
    word_occurrence_summary[word] = {
        'num_docs': num_docs,
        'total_occurrences': total_occurrences,
        'by_doc': doc_dict
    }

# Print the results for each word that appears in at least one document.
# (This prints all words; adjust the condition if you only want words appearing in multiple documents.)
for word, summary in word_occurrence_summary.items():
    # Uncomment the next line if you only want words that appear in more than one document:
    # if summary['num_docs'] < 2: continue
    print(f"Word: '{word}'")
    print(f"  Appears in {summary['num_docs']} document(s) with a total of {summary['total_occurrences']} occurrences.")
    for doc, count in summary['by_doc'].items():
        print(f"    {doc}: {count}")
    print()


Word: 'artificial'
  Appears in 5 document(s) with a total of 76 occurrences.
    artificial_intelligence.txt: 45
    chatgpt_wikipedia.txt: 8
    computer_science.txt: 6
    elon_musk.txt: 7
    nvidia.txt: 10

Word: 'intelligence'
  Appears in 5 document(s) with a total of 103 occurrences.
    artificial_intelligence.txt: 65
    chatgpt_wikipedia.txt: 13
    computer_science.txt: 8
    elon_musk.txt: 7
    nvidia.txt: 10

Word: 'ai'
  Appears in 5 document(s) with a total of 309 occurrences.
    artificial_intelligence.txt: 227
    chatgpt_wikipedia.txt: 57
    computer_science.txt: 2
    elon_musk.txt: 4
    nvidia.txt: 19

Word: 'refers'
  Appears in 1 document(s) with a total of 1 occurrences.
    artificial_intelligence.txt: 1

Word: 'capability'
  Appears in 2 document(s) with a total of 2 occurrences.
    artificial_intelligence.txt: 1
    nvidia.txt: 1

Word: 'computational'
  Appears in 3 document(s) with a total of 22 occurrences.
    artificial_intelligence.txt: 7
    compu

In [19]:
import math

# Total number of documents
total_docs = len(filtered_data)

# Compute total token counts for each document from the filtered tokens
doc_token_counts = {doc: len(tokens) for doc, tokens in filtered_data.items()}

# If not already computed, re-calculate the word occurrence summary from the previous analysis
word_occurrence_summary = {}
for word, doc_dict in word_document_counts.items():
    num_docs = len(doc_dict)
    total_occurrences = sum(doc_dict.values())
    word_occurrence_summary[word] = {
        'num_docs': num_docs,
        'total_occurrences': total_occurrences,
        'by_doc': doc_dict
    }

# Compute TF, DF, IDF (using 1 + log(total_docs / DF)), and TF-IDF for each word
tf_idf_results = {}
for word, summary in word_occurrence_summary.items():
    df = summary['num_docs']
    # Updated IDF calculation: IDF = 1 + log(total_docs / DF)
    idf = 1 + math.log(total_docs / df) if df else 0
    
    # Compute TF and TF-IDF per document for this word
    tf_idf_per_doc = {}
    for doc, count in summary['by_doc'].items():
        tf = count / doc_token_counts[doc]  # Relative frequency in the document
        tf_idf = tf * idf
        tf_idf_per_doc[doc] = {
            'tf': tf,
            'tf_idf': tf_idf,
            'count': count,
            'doc_total': doc_token_counts[doc]
        }
    
    tf_idf_results[word] = {
        'df': df,
        'idf': idf,
        'total_occurrences': summary['total_occurrences'],
        'tf_idf_per_doc': tf_idf_per_doc
    }

# Display the TF, DF, IDF, and TF-IDF results for each word
for word, metrics in tf_idf_results.items():
    print(f"Word: '{word}'")
    print(f"  Document Frequency (DF): {metrics['df']} out of {total_docs} documents")
    print(f"  Inverse Document Frequency (IDF): {metrics['idf']:.4f}")
    for doc, values in metrics['tf_idf_per_doc'].items():
        print(f"    {doc}: count = {values['count']}, TF = {values['tf']:.4f}, TF-IDF = {values['tf_idf']:.4f}")
    print()


Word: 'artificial'
  Document Frequency (DF): 5 out of 5 documents
  Inverse Document Frequency (IDF): 1.0000
    artificial_intelligence.txt: count = 45, TF = 0.0056, TF-IDF = 0.0056
    chatgpt_wikipedia.txt: count = 8, TF = 0.0016, TF-IDF = 0.0016
    computer_science.txt: count = 6, TF = 0.0026, TF-IDF = 0.0026
    elon_musk.txt: count = 7, TF = 0.0017, TF-IDF = 0.0017
    nvidia.txt: count = 10, TF = 0.0024, TF-IDF = 0.0024

Word: 'intelligence'
  Document Frequency (DF): 5 out of 5 documents
  Inverse Document Frequency (IDF): 1.0000
    artificial_intelligence.txt: count = 65, TF = 0.0081, TF-IDF = 0.0081
    chatgpt_wikipedia.txt: count = 13, TF = 0.0025, TF-IDF = 0.0025
    computer_science.txt: count = 8, TF = 0.0034, TF-IDF = 0.0034
    elon_musk.txt: count = 7, TF = 0.0017, TF-IDF = 0.0017
    nvidia.txt: count = 10, TF = 0.0024, TF-IDF = 0.0024

Word: 'ai'
  Document Frequency (DF): 5 out of 5 documents
  Inverse Document Frequency (IDF): 1.0000
    artificial_intelligence

In [22]:
import math
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# --- Build Document Vectors from TF-IDF Results ---
# (Assumes tf_idf_results from Cell 9 and filtered_data are already defined)
doc_vectors = {doc: {} for doc in filtered_data.keys()}
for word, metrics in tf_idf_results.items():
    for doc, values in metrics['tf_idf_per_doc'].items():
        doc_vectors[doc][word] = values['tf_idf']

# Helper functions for vector operations
def dot_product(vec1, vec2):
    return sum(vec1.get(k, 0) * vec2.get(k, 0) for k in vec1)

def vector_norm(vec):
    return math.sqrt(sum(v**2 for v in vec.values()))

def cosine_similarity(vec1, vec2):
    norm1 = vector_norm(vec1)
    norm2 = vector_norm(vec2)
    if norm1 == 0 or norm2 == 0:
        return 0.0
    return dot_product(vec1, vec2) / (norm1 * norm2)

# --- Process the Query "machine learning" ---
query = "machine learning"
# Tokenize and preprocess the query (tokenization, lowercasing, filtering)
query_tokens = word_tokenize(query)
query_tokens = [word.lower() for word in query_tokens]
stop_words = set(stopwords.words('english'))
query_tokens = [word for word in query_tokens if word.isalnum() and word not in stop_words]

# Build query TF (using raw counts) and then normalize
query_tf = {}
for token in query_tokens:
    query_tf[token] = query_tf.get(token, 0) + 1
total_query_tokens = len(query_tokens)
query_tf = {token: count / total_query_tokens for token, count in query_tf.items()}

# Build query vector using the IDF values from tf_idf_results (if available, else idf=1)
query_vector = {}
for token, tf in query_tf.items():
    idf = 1.0
    if token in tf_idf_results:
        idf = tf_idf_results[token]['idf']
    query_vector[token] = tf * idf

print("Original Query Vector:", query_vector)

# --- Compute Relevance Scores (Cosine Similarity) for Each Document ---
relevance_scores = {}
for doc, vec in doc_vectors.items():
    score = cosine_similarity(query_vector, vec)
    relevance_scores[doc] = score
    print(f"Document: {doc}, Relevance Score: {score:.4f}")

# --- Pseudo Relevance Feedback via Rocchio Algorithm ---
# Parameters: α = 1.0, β = 0.75, γ = 0.15
alpha = 1.0
beta = 0.75
gamma = 0.15

# Select top 2 documents as relevant (pseudo-relevant)
sorted_docs = sorted(relevance_scores.items(), key=lambda x: x[1], reverse=True)
num_relevant = 2
relevant_docs = [doc for doc, score in sorted_docs[:num_relevant]]
non_relevant_docs = [doc for doc in doc_vectors.keys() if doc not in relevant_docs]

print("Relevant Documents for Feedback:", relevant_docs)
print("Non-Relevant Documents for Feedback:", non_relevant_docs)

# Function to compute the centroid (average vector) of a list of document vectors
def average_vector(doc_list):
    avg_vec = {}
    if not doc_list:
        return avg_vec
    for doc in doc_list:
        vec = doc_vectors[doc]
        for word, weight in vec.items():
            avg_vec[word] = avg_vec.get(word, 0) + weight
    for word in avg_vec:
        avg_vec[word] /= len(doc_list)
    return avg_vec

centroid_relevant = average_vector(relevant_docs)
centroid_non_relevant = average_vector(non_relevant_docs)

# Rocchio formula:
# new_query_vector = α * original_query_vector + β * (centroid_relevant) - γ * (centroid_non_relevant)
new_query_vector = {}
# Start with original query vector scaled by α
for word, weight in query_vector.items():
    new_query_vector[word] = alpha * weight

# Add β * centroid_relevant
for word, weight in centroid_relevant.items():
    new_query_vector[word] = new_query_vector.get(word, 0) + beta * weight

# Subtract γ * centroid_non_relevant
for word, weight in centroid_non_relevant.items():
    new_query_vector[word] = new_query_vector.get(word, 0) - gamma * weight
    # Set any negative weights to 0
    if new_query_vector[word] < 0:
        new_query_vector[word] = 0

Original Query Vector: {'machine': 0.5, 'learning': 0.5}
Document: artificial_intelligence.txt, Relevance Score: 0.1643
Document: chatgpt_wikipedia.txt, Relevance Score: 0.0359
Document: computer_science.txt, Relevance Score: 0.0460
Document: elon_musk.txt, Relevance Score: 0.0045
Document: nvidia.txt, Relevance Score: 0.0229
Relevant Documents for Feedback: ['artificial_intelligence.txt', 'computer_science.txt']
Non-Relevant Documents for Feedback: ['chatgpt_wikipedia.txt', 'elon_musk.txt', 'nvidia.txt']
Expanded Query Vector (after Rocchio): {'machine': 0.5034487207260767, 'learning': 0.5029495356909358, 'artificial': 0.002782505259318033, 'intelligence': 0.003986902722382605, 'ai': 0.010047958531205773, 'refers': 0.00012133158303320369, 'capability': 6.64777250638266e-05, 'computational': 0.0038942753498130917, 'systems': 0.004464429259292782, 'perform': 0.0002282460065430871, 'tasks': 0.0015611316128321636, 'typically': 0.0006899007277265805, 'associated': 0.0004656331208904476, 'h

In [21]:
import math
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# --- Build Document Vectors from TF-IDF Results ---
doc_vectors = {doc: {} for doc in filtered_data.keys()}
for word, metrics in tf_idf_results.items():
    for doc, values in metrics['tf_idf_per_doc'].items():
        doc_vectors[doc][word] = values['tf_idf']

def dot_product(vec1, vec2):
    return sum(vec1.get(k, 0) * vec2.get(k, 0) for k in vec1)

def vector_norm(vec):
    return math.sqrt(sum(v**2 for v in vec.values()))

def cosine_similarity(vec1, vec2):
    norm1, norm2 = vector_norm(vec1), vector_norm(vec2)
    return dot_product(vec1, vec2) / (norm1 * norm2) if norm1 and norm2 else 0.0

# --- Process the Query "machine learning" ---
query = "machine learning"
query_tokens = [w.lower() for w in word_tokenize(query) if w.isalnum() and w.lower() not in set(stopwords.words('english'))]

query_tf = {}
for token in query_tokens:
    query_tf[token] = query_tf.get(token, 0) + 1
total_query_tokens = len(query_tokens)
query_tf = {token: count / total_query_tokens for token, count in query_tf.items()}

query_vector = {}
for token, tf in query_tf.items():
    idf = tf_idf_results[token]['idf'] if token in tf_idf_results else 1.0
    query_vector[token] = tf * idf

# --- Compute Relevance Scores for Each Document ---
relevance_scores = {doc: cosine_similarity(query_vector, vec) for doc, vec in doc_vectors.items()}

# Select top 2 documents as pseudo-relevant
sorted_docs = sorted(relevance_scores.items(), key=lambda x: x[1], reverse=True)
relevant_docs = [doc for doc, score in sorted_docs[:2]]
non_relevant_docs = [doc for doc in doc_vectors.keys() if doc not in relevant_docs]

def average_vector(doc_list):
    avg = {}
    if not doc_list:
        return avg
    for doc in doc_list:
        for word, weight in doc_vectors[doc].items():
            avg[word] = avg.get(word, 0) + weight
    for word in avg:
        avg[word] /= len(doc_list)
    return avg

centroid_relevant = average_vector(relevant_docs)
centroid_non_relevant = average_vector(non_relevant_docs)

# --- Rocchio Algorithm to Expand Query ---
alpha, beta, gamma = 1.0, 0.75, 0.15
new_query_vector = {w: alpha * weight for w, weight in query_vector.items()}
for word, weight in centroid_relevant.items():
    new_query_vector[word] = new_query_vector.get(word, 0) + beta * weight
for word, weight in centroid_non_relevant.items():
    new_query_vector[word] = max(0, new_query_vector.get(word, 0) - gamma * weight)

# --- Construct a Short Expanded Query from Top 5 Terms ---
top_terms = sorted(new_query_vector.items(), key=lambda x: x[1], reverse=True)[:5]
expanded_query = " ".join([term for term, weight in top_terms if weight > 0])

print("Original Query:", query)
print("Relevant Documents:", relevant_docs)
print("Expanded Query:", expanded_query)


Original Query: machine learning
Relevant Documents: ['artificial_intelligence.txt', 'computer_science.txt']
Expanded Query: machine learning computer science ai
